In [ ]:
import requests

url = 'https://raw.githubusercontent.com/EinarOlafsson/spacr/main/code/spacr_data_generation.py'
response = requests.get(url)
with open('spacr_data_generation.py', 'w') as file:
    file.write(response.text)
    
import spacr_data_generation as spacr

In [ ]:
# 1. Organize images into npy stacks.
# 2. Preprocess images for object detection.
# 3. Generate masks.

settings = {'metadata_type':'cellvoyager',
            'experiment':'lovastatin_tsg101',
            'channels':[0,1,2,3],
            'nucleus_channel':0,
            'nucleus_background':500,
            'nucleus_Signal_to_noise':5,
            'nucleus_CP_prob':0,
            'cell_channel':3,
            'cell_background':200,
            'cell_Signal_to_noise':5,
            'cell_CP_prob':0,
            'pathogen_channel':2,
            'pathogen_background':100,
            'pathogen_Signal_to_noise':3,
            'pathogen_CP_prob':-1,
            'magnefication':20}

advanced_settings = {'preprocess':True,
                    'masks':True,
                    'custom_regex':None,
                    'save':True,
                    'plot':False,
                    'normalize_plots':True,
                    'examples_to_plot':1,
                    'batch_size':100,
                    'randomize':True,
                    'remove_background':True,
                    'lower_quantile':0.02,
                    'merge':False,
                    'all_to_mip':False,
                    'pick_slice':False,
                    'skip_mode':None,
                    'timelapse':False,
                    'fps':2,
                    'workers':30,
                    'verbose':False}

src = 'path'
spacr.preprocess_generate_masks(src,settings=settings,advanced_settings=advanced_settings)

In [ ]:
# Visualize masks and images

plot_settings = {'include_noninfected':False,
                 'include_multiinfected':True,
                 'include_multinucleated':True,
                 'filter_min_max':[[0,100000],[0,100000],[0,100000],[0,100000]],
                 'channel_dims':[0,1,2,3],
                 'remove_background':False,
                 'backgrounds':[100,100,100,100],
                 'cell_mask_dim':4,
                 'nucleus_mask_dim':None,
                 'pathogen_mask_dim':6,
                 'overlay_chans':[0,2,3],
                 'outline_thickness':2,
                 'outline_color':'gbr',
                 'overlay_chans':[3,2,0],
                 'overlay':True,
                 'normalization_percentiles':[1,99],
                 'normalize':True,
                 'print_object_number':False,
                 'nr':1,
                 'figuresize':20,
                 'cmap':'inferno',
                 'verbose':False}

src = 'path'
spacr.plot_merged(src, plot_settings)

In [ ]:
# 1. Collect measurement data and save to database
# 2. Generate single object images.

src = 'path'

if __name__ == "__main__":
    settings = {'experiment':'lovastatin_hela',
                'input_folder':src+'/merged',
                'channels':[0,1,2,3],
                'cell_mask_dim':4,
                'cell_min_size':0,
                'nuclei_mask_dim':5,
                'nucleus_min_size':0,
                'pathogen_mask_dim':6,
                'pathogen_min_size':0,
                'cytoplasm_min_size':0,
                'save_png':True,
                'crop_mode':['cell'],
                'use_bounding_box':False,
                'png_size':[[224,224]],
                'normalize':[1,99],
                'png_dims':[0,1,3],
                'normalize_by':'png', #'fov' or 'png'
                'save_measurements':True,
                'plot':False,
                'plot_filtration':False,
                'include_uninfected':False,
                'dialate_pngs':False,
                'dialate_png_ratios':[0.2],
                'max_workers':30}
    
    spacr.measure_crop(settings)

In [ ]:
# Annotate single object images.
src = 'path'

spacr.annotate(db=src+'/measurements/measurements.db',
         image_type="cell_png", 
         channels=['r','g','b'],
         annotation_column='test',
         geom="3200x2000", 
         img_size=(200, 200), 
         rows=10, 
         columns=18)

In [ ]:
# Calculate recruitment.

metadata_settings = {'target':'TSG101',
                     'cell_types':['HeLa'],
                     'cell_plate_metadata':None,
                     'pathogen_types':['Dsag1', 'Dgra14_1', 'Dgra14_2', 'Dgra14_3'],
                     'pathogen_plate_metadata':[['c1', 'c7'],['c3', 'c8'],['c3','c4','c9','c10'],['c5','c6','c11','c12']],
                     'treatments':['cm', 'lova'],
                     'treatment_plate_metadata':[['c1','c2','c3','c4','c5','c6'],['c7','c8','c9','c10','c11','c12']],
                     'metadata_types':'col',
                     'channel_dims':[0,1,2,3],
                     'cell_chann_dim':3,
                     'cell_mask_dim':4,
                     'nucleus_chann_dim':0,
                     'nucleus_mask_dim':5,
                     'pathogen_chann_dim':2,
                     'pathogen_mask_dim':6,
                     'channel_of_interest':1}

advanced_settings = {'plot':False,
                     'plot_nr':1,
                     'plot_control':True,
                     'figuresize':20,
                     'remove_background':True,
                     'backgrounds':100,
                     'include_noninfected':False,
                     'include_multiinfected':True,
                     'include_multinucleated':True,
                     'cells_per_well':10,
                     'pathogen_size_range':[500,2500],
                     'nucleus_size_range':[1000,5000],
                     'cell_size_range':[5000,10000],
                     'pathogen_intensity_range':[500,5000],
                     'nucleus_intensity_range':[500,5000],
                     'cell_intensity_range':[500,5000],
                     'target_intensity_min':800}

src = 'path'
dfs = spacr.analyze_recruitment(src,metadata_settings,advanced_settings)